## HW1. Проектирование API
Автор: *Лаврухина Виктория Вячеславовна*, 19.10.2025

Это задание выполняется в рамках модуля 1 «Проектирование API». Вы закрепите навыки разработки API, используя подход сode-first, затем будете придерживаться подхода API-first

> Чтобы получить максимальный балл, убедитесь, что ваш ноутбук запускается с нуля, структура понятна, а в выводах вы объясняете свои решения.  

## Подготовка окружения

In [17]:
!pip install fastapi[all] uvicorn["standard"]


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


Теперь, когда у нас установлены необходимые библиотеки, мы можем приступить к созданию нашего первого приложения FastAPI.

### Задание 1

Вам необходимо самостоятельно создать веб-сервер на основе FastAPI.

Задача: создать файл `main.py`, который будет содержать наш код API с четырьмя методами HTTP.



In [18]:
%%writefile main.py
from typing import Dict, Optional
from fastapi import FastAPI, HTTPException, Path, Body, Response
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field

app = FastAPI(
    title="API Design HW",
    version="0.1.0",
    description=(
        "Минимальный сервер FastAPI для ДЗ: 4 HTTP-метода (GET/POST/PUT/DELETE) "
        "для ресурса /items и /health. Используются JSONResponse и HTTPException."
    ),
)

class Item(BaseModel):
    name: str = Field(..., example="Widget")
    price: float = Field(..., ge=0.0, example=19.99)
    in_stock: bool = Field(True, example=True)
    description: Optional[str] = Field(None, example="Short description")

DB: Dict[int, Item] = {}
COUNTER: int = 0


@app.get("/health", tags=["utility"], summary="Health check", response_description="OK")
def health() -> JSONResponse:
    """Быстрый статус сервера. Возвращает JSONResponse."""
    return JSONResponse(status_code=200, content={"status": "ok"})


@app.get(
    "/items/{item_id}",
    tags=["items"],
    summary="Получить товар по ID",
    response_model=Item,
    responses={404: {"description": "Item not found"}},
)
def get_item(item_id: int = Path(..., ge=1, description="ID товара (>=1)")) -> Item:
    item = DB.get(item_id)
    if not item:
        raise HTTPException(status_code=404, detail=f"Item {item_id} not found")
    return item


@app.post(
    "/items",
    tags=["items"],
    summary="Создать товар",
    status_code=201,
    response_model=Item,
)
def create_item(item: Item = Body(..., description="Новый товар")) -> Item:
    global COUNTER
    COUNTER += 1
    DB[COUNTER] = item
    return item


@app.put(
    "/items/{item_id}",
    tags=["items"],
    summary="Обновить товар по ID",
    response_model=Item,
    responses={404: {"description": "Item not found"}},
)
def update_item(
    item_id: int = Path(..., ge=1, description="ID товара"),
    item: Item = Body(...),
) -> Item:
    if item_id not in DB:
        raise HTTPException(status_code=404, detail=f"Item {item_id} not found")
    DB[item_id] = item
    return item


@app.delete(
    "/items/{item_id}",
    tags=["items"],
    summary="Удалить товар по ID",
    status_code=204,
    responses={404: {"description": "Item not found"}, 204: {"description": "Deleted"}},
)
def delete_item(item_id: int = Path(..., ge=1, description="ID товара")) -> Response:
    if item_id not in DB:
        raise HTTPException(status_code=404, detail=f"Item {item_id} not found")
    del DB[item_id]
    return Response(status_code=204)


Overwriting main.py


#### Для проверки работы веб-сервера выполните следующие шаги:
1) Установите необходимые библиотеки ```pip install "fastapi[all]" "uvicorn[standard]"```
2) Выполните команду в терминале
```uvicorn main:app --reload --port 8000```
3) Теперь можно открыть браузер и перейти по следующим ссылкам:  
Swagger UI (интерактивная документация): http://127.0.0.1:8000/docs  
Health-check: http://127.0.0.1:8000/health

### Задание 2

Сервер должен отвечать валидным JSON на эндпоинте /json_data.

Задача: создать новый эндпоинт /json_data и подключить компонент JSONResponse на этом эндпоинте. Содержание JSONa не важно, главное, чтобы он был валидным.


In [19]:
%%writefile -a main.py
from fastapi.responses import JSONResponse

@app.get("/json_data", tags=["utility"], summary="Возврат валидного JSON")
def json_data():
    payload = {
        "status": "ok",
        "task": 2,
        "note": "Пример валидного JSON через JSONResponse",
        "data": {"a": 1, "b": [1, 2, 3]}
    }
    return JSONResponse(status_code=200, content=payload)

Appending to main.py


### Задание 3

Обработка ошибок с использованием простого HTTPException.

Задачи:
1. Изучите стандартные коды ошибок.
2. Выберите любое случайное число в диапазоне 400—526.

> Чтобы возвращать ошибки с соответствующими HTTP-статусами вам нужно:
> 1. подключить класс `HTTPException` и просто выдать любой [код ошибки](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP) (от 400 до 526);
> 2. создать маршрут /error, который будет генерировать ошибку с этим кодом в ответ на любой запрос.

В комментариях напишите, как вы понимаете, что означает выбранный вами код ошибки.

In [20]:
%%writefile -a main.py 
import random
from fastapi import HTTPException

@app.get("/error", tags=["utility"], summary="Генерация случайной ошибки HTTP")
def generate_error():
    """
    Эндпоинт /error возвращает случайный код ошибки (400–526).
    Используется HTTPException для демонстрации обработки ошибок.
    """
    error_code = random.randint(400, 526)

    # Пример описания значений
    descriptions = {
        400: "Bad Request — некорректный запрос клиента.",
        404: "Not Found — запрашиваемый ресурс не найден.",
        418: "I'm a teapot — шуточный код из RFC 2324.",
        500: "Internal Server Error — ошибка на сервере.",
        503: "Service Unavailable — сервис временно недоступен.",
    }

    # Подставляем краткое описание, если есть
    message = descriptions.get(error_code, "Случайная ошибка в диапазоне 400–526.")

    raise HTTPException(status_code=error_code, detail=message)

Appending to main.py


### Задание 4

Создание автодокументации для API: FastAPI автоматически генерирует документацию API в формате OpenAPI и предоставляет интерфейс Swagger UI для ее просмотра

*Поскольку код коллаба выполняется внутри виртуальной машины, у которой нет внешнего IP-адреса, вам потребуется создать тоннель, чтобы получить внешний IP-адрес.*

*Зарегистрируйтесь в личном кабинете https://xtunnel.ru/ и скопируйте бесплатную лицензию (секретный ключ API)*

Если возникают сложности, используйте локальную версию коллаба.
```bash
pip install notebook
jupyter notebook
```


Задача. Чтобы проверить документацию, выполните следующие шаги:
1. Запустите сервер FastAPI с помощью команды ниже.
2. Откройте браузер и перейдите по адресу `http://127.0.0.1:8000/docs` для просмотра документации в Swagger UI.
3. Для просмотра документации в формате OpenAPI перейдите по адресу `http://127.0.0.1:8000/openapi.json`

In [21]:
"""

{
  "openapi": "3.1.0",
  "info": {
    "title": "API Design HW",
    "description": "Минимальный сервер FastAPI для ДЗ: 4 HTTP-метода (GET/POST/PUT/DELETE) для ресурса /items и /health. Используются JSONResponse и HTTPException.",
    "version": "0.1.0"
  },
  "paths": {
    "/health": {
      "get": {
        "tags": [
          "utility"
        ],
        "summary": "Health check",
        "description": "Быстрый статус сервера. Возвращает JSONResponse.",
        "operationId": "health_health_get",
        "responses": {
          "200": {
            "description": "OK",
            "content": {
              "application/json": {
                "schema": {}
              }
            }
          }
        }
      }
    },
    "/items/{item_id}": {
      "get": {
        "tags": [
          "items"
        ],
        "summary": "Получить товар по ID",
        "operationId": "get_item_items__item_id__get",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer",
              "minimum": 1,
              "description": "ID товара (>=1)",
              "title": "Item Id"
            },
            "description": "ID товара (>=1)"
          }
        ],
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Item"
                }
              }
            }
          },
          "404": {
            "description": "Item not found"
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      },
      "put": {
        "tags": [
          "items"
        ],
        "summary": "Обновить товар по ID",
        "operationId": "update_item_items__item_id__put",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer",
              "minimum": 1,
              "description": "ID товара",
              "title": "Item Id"
            },
            "description": "ID товара"
          }
        ],
        "requestBody": {
          "required": true,
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/Item"
              }
            }
          }
        },
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Item"
                }
              }
            }
          },
          "404": {
            "description": "Item not found"
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      },
      "delete": {
        "tags": [
          "items"
        ],
        "summary": "Удалить товар по ID",
        "operationId": "delete_item_items__item_id__delete",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer",
              "minimum": 1,
              "description": "ID товара",
              "title": "Item Id"
            },
            "description": "ID товара"
          }
        ],
        "responses": {
          "204": {
            "description": "Deleted"
          },
          "404": {
            "description": "Item not found"
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      }
    },
    "/items": {
      "post": {
        "tags": [
          "items"
        ],
        "summary": "Создать товар",
        "operationId": "create_item_items_post",
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/Item",
                "description": "Новый товар"
              }
            }
          },
          "required": true
        },
        "responses": {
          "201": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Item"
                }
              }
            }
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      }
    },
    "/json_data": {
      "get": {
        "tags": [
          "utility"
        ],
        "summary": "Возврат валидного JSON",
        "operationId": "json_data_json_data_get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {}
              }
            }
          }
        }
      }
    },
    "/error": {
      "get": {
        "tags": [
          "utility"
        ],
        "summary": "Генерация случайной ошибки HTTP",
        "description": "Эндпоинт /error возвращает случайный код ошибки (400–526).\nИспользуется HTTPException для демонстрации обработки ошибок.",
        "operationId": "generate_error_error_get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {}
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "HTTPValidationError": {
        "properties": {
          "detail": {
            "items": {
              "$ref": "#/components/schemas/ValidationError"
            },
            "type": "array",
            "title": "Detail"
          }
        },
        "type": "object",
        "title": "HTTPValidationError"
      },
      "Item": {
        "properties": {
          "name": {
            "type": "string",
            "title": "Name",
            "example": "Widget"
          },
          "price": {
            "type": "number",
            "minimum": 0,
            "title": "Price",
            "example": 19.99
          },
          "in_stock": {
            "type": "boolean",
            "title": "In Stock",
            "default": true,
            "example": true
          },
          "description": {
            "anyOf": [
              {
                "type": "string"
              },
              {
                "type": "null"
              }
            ],
            "title": "Description",
            "example": "Short description"
          }
        },
        "type": "object",
        "required": [
          "name",
          "price"
        ],
        "title": "Item"
      },
      "ValidationError": {
        "properties": {
          "loc": {
            "items": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "integer"
                }
              ]
            },
            "type": "array",
            "title": "Location"
          },
          "msg": {
            "type": "string",
            "title": "Message"
          },
          "type": {
            "type": "string",
            "title": "Error Type"
          }
        },
        "type": "object",
        "required": [
          "loc",
          "msg",
          "type"
        ],
        "title": "ValidationError"
      }
    }
  }
}

"""

'\n\n{\n  "openapi": "3.1.0",\n  "info": {\n    "title": "API Design HW",\n    "description": "Минимальный сервер FastAPI для ДЗ: 4 HTTP-метода (GET/POST/PUT/DELETE) для ресурса /items и /health. Используются JSONResponse и HTTPException.",\n    "version": "0.1.0"\n  },\n  "paths": {\n    "/health": {\n      "get": {\n        "tags": [\n          "utility"\n        ],\n        "summary": "Health check",\n        "description": "Быстрый статус сервера. Возвращает JSONResponse.",\n        "operationId": "health_health_get",\n        "responses": {\n          "200": {\n            "description": "OK",\n            "content": {\n              "application/json": {\n                "schema": {}\n              }\n            }\n          }\n        }\n      }\n    },\n    "/items/{item_id}": {\n      "get": {\n        "tags": [\n          "items"\n        ],\n        "summary": "Получить товар по ID",\n        "operationId": "get_item_items__item_id__get",\n        "parameters": [\n         

### Задание 5

Автодокументаци API хороша для маленьких проектов. Сейчас вам нужно полученый в задании 4 JSON вставить в редактор [Swagger](https://editor.swagger.io/), добавить проектируемый маршрут с ответом в формате XML и сохранить описание в YAML.

Задачи:
1. Откройте редактор Swagger, вставьте JSON (ответьте ОК на запрос Would you like to convert your JSON into YAML?)
2. Добавьте 1 эндпоинт.
3. Скопируйте получившееся описание в YAML и вставьте в ячейку ниже.

In [22]:
"""

openapi: 3.1.0
info:
  title: API Design HW
  description: 'Минимальный сервер FastAPI для ДЗ: 4 HTTP-метода (GET/POST/PUT/DELETE) для ресурса /items и /health. Используются JSONResponse и HTTPException.'
  version: 0.1.0
paths:
  /health:
    get:
      tags:
        - utility
      summary: Health check
      description: Быстрый статус сервера. Возвращает JSONResponse.
      operationId: health_health_get
      responses:
        '200':
          description: OK
          content:
            application/json:
              schema: {}
  /status.xml:
    get:
      tags: [utility]
      summary: Статус сервиса (XML)
      description: Возвращает статус сервиса в формате XML.
      responses:
        '200':
          description: Пример XML-ответа
          content:
            application/xml:
              schema:
                type: object
                properties:
                  status:
                    type: string
                    example: ok
                  timestamp:
                    type: string
                    example: 2025-10-19T12:00:00Z
              example: |
                <ServiceStatus>
                  <status>ok</status>
                  <timestamp>2025-10-19T12:00:00Z</timestamp>
                </ServiceStatus>

  /items/{item_id}:
    get:
      tags:
        - items
      summary: Получить товар по ID
      operationId: get_item_items__item_id__get
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            minimum: 1
            description: ID товара (>=1)
            title: Item Id
          description: ID товара (>=1)
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Item'
        '404':
          description: Item not found
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    put:
      tags:
        - items
      summary: Обновить товар по ID
      operationId: update_item_items__item_id__put
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            minimum: 1
            description: ID товара
            title: Item Id
          description: ID товара
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Item'
        '404':
          description: Item not found
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    delete:
      tags:
        - items
      summary: Удалить товар по ID
      operationId: delete_item_items__item_id__delete
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            minimum: 1
            description: ID товара
            title: Item Id
          description: ID товара
      responses:
        '204':
          description: Deleted
        '404':
          description: Item not found
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /items:
    post:
      tags:
        - items
      summary: Создать товар
      operationId: create_item_items_post
      requestBody:
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
              description: Новый товар
        required: true
      responses:
        '201':
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Item'
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /json_data:
    get:
      tags:
        - utility
      summary: Возврат валидного JSON
      operationId: json_data_json_data_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
  /error:
    get:
      tags:
        - utility
      summary: Генерация случайной ошибки HTTP
      description: |-
        Эндпоинт /error возвращает случайный код ошибки (400–526).
        Используется HTTPException для демонстрации обработки ошибок.
      operationId: generate_error_error_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
components:
  schemas:
    HTTPValidationError:
      properties:
        detail:
          items:
            $ref: '#/components/schemas/ValidationError'
          type: array
          title: Detail
      type: object
      title: HTTPValidationError
    Item:
      properties:
        name:
          type: string
          title: Name
          example: Widget
        price:
          type: number
          minimum: 0
          title: Price
          example: 19.99
        in_stock:
          type: boolean
          title: In Stock
          default: true
          example: true
        description:
          anyOf:
            - type: string
            - type: 'null'
          title: Description
          example: Short description
      type: object
      required:
        - name
        - price
      title: Item
    ValidationError:
      properties:
        loc:
          items:
            anyOf:
              - type: string
              - type: integer
          type: array
          title: Location
        msg:
          type: string
          title: Message
        type:
          type: string
          title: Error Type
      type: object
      required:
        - loc
        - msg
        - type
      title: ValidationError


"""

"\n\nopenapi: 3.1.0\ninfo:\n  title: API Design HW\n  description: 'Минимальный сервер FastAPI для ДЗ: 4 HTTP-метода (GET/POST/PUT/DELETE) для ресурса /items и /health. Используются JSONResponse и HTTPException.'\n  version: 0.1.0\npaths:\n  /health:\n    get:\n      tags:\n        - utility\n      summary: Health check\n      description: Быстрый статус сервера. Возвращает JSONResponse.\n      operationId: health_health_get\n      responses:\n        '200':\n          description: OK\n          content:\n            application/json:\n              schema: {}\n  /status.xml:\n    get:\n      tags: [utility]\n      summary: Статус сервиса (XML)\n      description: Возвращает статус сервиса в формате XML.\n      responses:\n        '200':\n          description: Пример XML-ответа\n          content:\n            application/xml:\n              schema:\n                type: object\n                properties:\n                  status:\n                    type: string\n                

## Итоговое оформление


1. Подготовьте ноутбук в логичной структуре: написание кода → работа с JSON → обработка ошибок → API в YAML → итоги.  
2. В ячейках Markdown сформулируйте 5–8 предложений с выводами, когда стоит применять подход Code-first и почему стоит придерживаться подхода API-first.  



### Выводы по подходам Code-first и API-first

| Подход         | Что делаем сначала              | Документация                       | Когда применять                        |
| -------------- | ------------------------------- | ---------------------------------- | -------------------------------------- |
| **Code-first** | Пишем код API                   | Генерируется автоматически         | Прототипы, учебные проекты, MVP        |
| **API-first**  | Проектируем схему API (Swagger) | Используется как контракт для кода | Командная разработка, крупные продукты |

Подход **Code-first** удобно применять на ранних этапах разработки, когда команда небольшая, а API разрабатывается параллельно с основной бизнес-логикой.  
Он позволяет быстро запускать прототипы, не тратя время на формализацию спецификации, и автоматически генерировать документацию (например, в FastAPI через Swagger UI).  
Этот подход особенно полезен для внутренних сервисов и учебных проектов, где важна скорость итераций и простота внедрения изменений.

Однако при росте проекта и увеличении числа команд **лучше переходить к API-first**.  
API-first обеспечивает единый, согласованный контракт взаимодействия между командами (backend, frontend, mobile) ещё до начала реализации.  
Это повышает предсказуемость интеграций, снижает количество ошибок и облегчает тестирование совместимости.  
Кроме того, API-first позволяет использовать генерацию SDK, мок-серверов и автотестов из спецификации OpenAPI, что делает процесс разработки более формальным, масштабируемым и прозрачным.  
Таким образом, Code-first удобен для быстрых MVP и экспериментов, а API-first — для зрелых, многокомандных и интеграционных проектов.
